## World Development Indicators

The World Development Indicators (WDI) is the primary World Bank collection of development indicators, compiled from officially-recognized international sources. It presents the most current and accurate global development data available, and includes national, regional and global estimates.

A searchable thematic list of all indicators is available [here](https://data.worldbank.org/indicator?tab=all).

In this notebook we will be connecting to the database `indicators`, downloaded from [Kaggle](https://www.kaggle.com/datasets/psycon/world-development-indicators?select=indicators.sqlite), which contains the 2022 World Development Indicators.

We will be using IPython's SQL extension and `%sql`/`%%sql` magic functions to query this database. 

In [1]:
# Load IPython SQL extension 
%load_ext sql

# Connect to the `indicators` database.
%sql sqlite:///indicators.sqlite

Then we want to list all tables in this database. This can be done by using a `SELECT` statement on `sqlite_master`. `sqlite_master` is an internal table present in all SQLite databases that describes the database's schema.

In [2]:
%%sql

SELECT *
    FROM sqlite_master
    WHERE type='table'

 * sqlite:///indicators.sqlite
Done.


type,name,tbl_name,rootpage,sql
table,Country,Country,2,"CREATE TABLE Country ( CountryCode TEXT, ShortName TEXT, TableName TEXT, LongName TEXT, Alpha2Code TEXT, CurrencyUnit TEXT, SpecialNotes TEXT, Region TEXT, IncomeGroup TEXT, Wb2Code TEXT, NationalAccountsBaseYear TEXT, NationalAccountsReferenceYear TEXT, SnaPriceValuation TEXT, LendingCategory TEXT, OtherGroups TEXT, SystemOfNationalAccounts TEXT, AlternativeConversionFactor TEXT, PppSurveyYear TEXT, BalanceOfPaymentsManualInUse TEXT, ExternalDebtReportingStatus TEXT, SystemOfTrade TEXT, GovernmentAccountingConcept TEXT, ImfDataDisseminationStandard TEXT, LatestPopulationCensus TEXT, LatestHouseholdSurvey TEXT, SourceOfMostRecentIncomeAndExpenditureData TEXT, VitalRegistrationComplete TEXT, LatestAgriculturalCensus TEXT, LatestIndustrialData NUMERIC, LatestTradeData NUMERIC, LatestWaterWithdrawalData NUMERIC)"
table,CountryNotes,CountryNotes,186,"CREATE TABLE CountryNotes ( Countrycode TEXT, Seriescode TEXT, Description TEXT)"
table,Series,Series,948,"CREATE TABLE Series ( SeriesCode TEXT, Topic TEXT, IndicatorName TEXT, ShortDefinition TEXT, LongDefinition TEXT, UnitOfMeasure TEXT, Periodicity TEXT, BasePeriod TEXT, OtherNotes NUMERIC, AggregationMethod TEXT, LimitationsAndExceptions TEXT, NotesFromOriginalSource TEXT, GeneralComments TEXT, Source TEXT, StatisticalConceptAndMethodology TEXT, DevelopmentRelevance TEXT, RelatedSourceLinks TEXT, OtherWebLinks NUMERIC, RelatedIndicators NUMERIC, LicenseType TEXT)"
table,Indicators,Indicators,4448,"CREATE TABLE Indicators ( CountryName TEXT, CountryCode TEXT, IndicatorName TEXT, IndicatorCode TEXT, Year INTEGER, Value NUMERIC)"
table,SeriesNotes,SeriesNotes,1317550,"CREATE TABLE SeriesNotes ( Seriescode TEXT, Year TEXT, Description TEXT)"
table,Footnotes,Footnotes,1317587,"CREATE TABLE Footnotes ( Countrycode TEXT, Seriescode TEXT, Year TEXT, Description TEXT)"


We have 6 tables available: `Country`, `CountryNotes`, `Series`, `Indicators`, `SeriesNotes` and `Footnotes`.

## Initial Exploration

In [3]:
%%sql

SELECT *
    FROM Country
    LIMIT 5

 * sqlite:///indicators.sqlite
Done.


CountryCode,ShortName,TableName,LongName,Alpha2Code,CurrencyUnit,SpecialNotes,Region,IncomeGroup,Wb2Code,NationalAccountsBaseYear,NationalAccountsReferenceYear,SnaPriceValuation,LendingCategory,OtherGroups,SystemOfNationalAccounts,AlternativeConversionFactor,PppSurveyYear,BalanceOfPaymentsManualInUse,ExternalDebtReportingStatus,SystemOfTrade,GovernmentAccountingConcept,ImfDataDisseminationStandard,LatestPopulationCensus,LatestHouseholdSurvey,SourceOfMostRecentIncomeAndExpenditureData,VitalRegistrationComplete,LatestAgriculturalCensus,LatestIndustrialData,LatestTradeData,LatestWaterWithdrawalData
AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period for national accounts data: FY (from 2013 are CY). National accounts data are sourced from the IMF and differ from the Central Statistics Organization numbers due to exclusion of the opium economy.,South Asia,Low income,AF,2002/03,,Value added at basic prices (VAB),IDA,HIPC,Country uses the 1993 System of National Accounts methodology.,,,,Actual,General trade system,Consolidated central government,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",,2013/14,,2013,2000
ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,,Europe & Central Asia,Upper middle income,AL,Original chained constant price data are rescaled.,1996,Value added at basic prices (VAB),IBRD,,Country uses the 1993 System of National Accounts methodology.,,Rolling,"IMF Balance of Payments Manual, 6th edition.",Actual,General trade system,Budgetary central government,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09","Living Standards Measurement Study Survey (LSMS), 2011/12",Yes,2012,2011,2013,2006
DZA,Algeria,Algeria,People's Democratic Republic of Algeria,DZ,Algerian dinar,,Middle East & North Africa,Upper middle income,DZ,1980,,Value added at basic prices (VAB),IBRD,,Country uses the 1968 System of National Accounts methodology.,,2011,"IMF Balance of Payments Manual, 6th edition.",Actual,Special trade system,Budgetary central government,General Data Dissemination System (GDDS),2008,"Multiple Indicator Cluster Survey (MICS), 2012","Integrated household survey (IHS), 1995",,,2010,2013,2001
ASM,American Samoa,American Samoa,American Samoa,AS,U.S. dollar,,East Asia & Pacific,Upper middle income,AS,,,,,,Country uses the 1968 System of National Accounts methodology.,,2011 (household consumption only).,,,Special trade system,,,2010,,,Yes,2007,,,
ADO,Andorra,Andorra,Principality of Andorra,AD,Euro,,Europe & Central Asia,High income: nonOECD,AD,2000,,Value added at basic prices (VAB),,,Country uses the 1968 System of National Accounts methodology.,,,,,Special trade system,,,2011. Population data compiled from administrative registers.,,,Yes,,,2006,


In [4]:
%%sql

SELECT Region, count(distinct TableName)
    FROM Country
GROUP BY 1 
ORDER BY 2 DESC

 * sqlite:///indicators.sqlite
Done.


Region,count(distinct TableName)
Europe & Central Asia,57
Sub-Saharan Africa,48
Latin America & Caribbean,41
East Asia & Pacific,36
,33
Middle East & North Africa,21
South Asia,8
North America,3


There's seems to be a `Region` that's missing a label, let's investigate what's in there.

In [5]:
%%sql

SELECT distinct TableName
    FROM Country
WHERE Region=''

 * sqlite:///indicators.sqlite
Done.


TableName
Arab World
Caribbean small states
Central Europe and the Baltics
East Asia & Pacific (all income levels)
East Asia & Pacific
Euro area
Europe & Central Asia (all income levels)
Europe & Central Asia
European Union
Fragile and conflict affected situations


There are different aggregations of countries (World, Lower middle income, European Union, etc.) under the unlabeled `Region`. This might be useful later in our analysis.

In [6]:
%%sql


    SELECT count(distinct IndicatorName)
FROM Indicators 


 * sqlite:///indicators.sqlite
Done.


count(distinct IndicatorName)
1344


There are 1344 different indicators in the `Indicators` table. For the sake of readability it's best to not display them all here. As previously mentioned, the full list of indicators is available [here](https://data.worldbank.org/indicator?tab=all).

Let's begin by exploring the ones that relate to **population growth**.

### Population growth

In [7]:
%%sql

SELECT distinct IndicatorName, IndicatorCode
    FROM indicators
    WHERE lower(IndicatorName) like '%population%' and lower(IndicatorName) like '%growth%'

 * sqlite:///indicators.sqlite
Done.


IndicatorName,IndicatorCode
"Annualized average growth rate in per capita real survey mean consumption or income, bottom 40% of population (%)",SI.SPR.PC40.ZG
"Annualized average growth rate in per capita real survey mean consumption or income, total population (%)",SI.SPR.PCAP.ZG
Population growth (annual %),SP.POP.GROW
Rural population growth (annual %),SP.RUR.TOTL.ZG
Urban population growth (annual %),SP.URB.GROW


Let's find the **average annual % of population growth** for each region in the **first** and **last year** of the dataset. 

In [8]:
%%sql

SELECT min(Year), max(Year)
FROM Indicators 
WHERE IndicatorName='Population growth (annual %)'
AND Value is not null

 * sqlite:///indicators.sqlite
Done.


min(Year),max(Year)
1960,2014


In [9]:
%%sql

with avg_1960 as (
    SELECT Year, Region, AVG(Value) as avg_pop_growth_1960
FROM Indicators i
INNER JOIN Country c
ON i.CountryCode=c.CountryCode
AND IndicatorName='Population growth (annual %)'
AND Year='1960'
GROUP BY 1, 2
ORDER BY 3 DESC)
,

avg_2014 as (
    SELECT Year, Region, AVG(Value) as avg_pop_growth_2014
FROM Indicators i
INNER JOIN Country c
ON i.CountryCode=c.CountryCode
AND IndicatorName='Population growth (annual %)'
AND Year='2014'
GROUP BY 1, 2
ORDER BY 3 DESC)

SELECT a.region, avg_pop_growth_1960, avg_pop_growth_2014
FROM avg_1960 a
LEFT JOIN avg_2014 b
ON a.Region=b.Region


 * sqlite:///indicators.sqlite
Done.


Region,avg_pop_growth_1960,avg_pop_growth_2014
Middle East & North Africa,3.780201152913741,2.139176439837713
East Asia & Pacific,2.677892617800957,1.1398734361736846
Latin America & Caribbean,2.351085690872689,0.9647484633748492
South Asia,2.2612439102671047,1.6162547934178277
Sub-Saharan Africa,2.1580339120058922,2.5284050617564304
North America,1.7870752166673736,0.7002142257910567
Europe & Central Asia,1.666259013801305,0.30974567864718655


How much did the annual % of Population growth change from 1960 to 2014?

In [10]:
%%sql

with avg_1960 as (
    SELECT Year, Region, AVG(Value) as avg_pop_growth_1960
FROM Indicators i
INNER JOIN Country c
ON i.CountryCode=c.CountryCode
AND IndicatorName='Population growth (annual %)'
AND Year='1960'
GROUP BY 1, 2
ORDER BY 3 DESC)
,

avg_2014 as (
    SELECT Year, Region, AVG(Value) as avg_pop_growth_2014
FROM Indicators i
INNER JOIN Country c
ON i.CountryCode=c.CountryCode
AND IndicatorName='Population growth (annual %)'
AND Year='2014'
GROUP BY 1, 2
ORDER BY 3 DESC)

SELECT a.Region, avg_pop_growth_1960, avg_pop_growth_2014, avg_pop_growth_2014-avg_pop_growth_1960 as change
FROM avg_1960 a
LEFT JOIN avg_2014 b
ON a.Region=b.Region
ORDER BY 4 DESC



 * sqlite:///indicators.sqlite
Done.


Region,avg_pop_growth_1960,avg_pop_growth_2014,change
Sub-Saharan Africa,2.1580339120058922,2.5284050617564304,0.3703711497505382
South Asia,2.2612439102671047,1.6162547934178277,-0.644989116849277
North America,1.7870752166673736,0.7002142257910567,-1.0868609908763167
Europe & Central Asia,1.666259013801305,0.30974567864718655,-1.3565133351541183
Latin America & Caribbean,2.351085690872689,0.9647484633748492,-1.3863372274978398
East Asia & Pacific,2.677892617800957,1.1398734361736846,-1.5380191816272724
Middle East & North Africa,3.780201152913741,2.139176439837713,-1.6410247130760283


All regions have suffered a **decrease** in their annual % of population growth except for Sub-Saharan Africa, with actually saw that number grow 0.37%. The highest drop in the annual % of Population growth between 1960 and 2014 occured for Middle East & North Africa, followed by East Asia & Pacific.

In [11]:
%%sql


    SELECT distinct IndicatorName
FROM Indicators 
WHERE lower(IndicatorName) like '%mortality%'


 * sqlite:///indicators.sqlite
Done.


IndicatorName
"Maternal mortality ratio (modeled estimate, per 100,000 live births)"
"Maternal mortality ratio (national estimate, per 100,000 live births)"
"Mortality rate, adult, female (per 1,000 female adults)"
"Mortality rate, adult, male (per 1,000 male adults)"
"Mortality rate, infant (per 1,000 live births)"
"Mortality rate, infant, female (per 1,000 live births)"
"Mortality rate, infant, male (per 1,000 live births)"
"Mortality rate, neonatal (per 1,000 live births)"
"Mortality rate, under-5 (per 1,000)"
"Mortality rate, under-5, female (per 1,000 live births)"


According to the [Pew Research Centre](https://www.pewresearch.org/religion/2015/04/02/main-factors-driving-population-growth/#:~:text=When%20demographers%20attempt%20to%20forecast,to%20begin%20with), the size of a population typically depends on four main factors: 
- Fertility rates
- Mortality rates (life expectancy)
- The initial age profile of the population (whether it is relatively old or relatively young to begin with)
- Migration

Let's explore indicators on mortality to determine whether they correlate with population growth.

### Mortality

In [12]:
%%sql


    SELECT min(Year), max(Year)
FROM Indicators 
WHERE IndicatorName='Mortality rate, under-5 (per 1,000)'
AND Value is not null


 * sqlite:///indicators.sqlite
Done.


min(Year),max(Year)
1960,2015


What year represented the highest mortality rate for children under 5 for each region?

In [13]:
%%sql

with all_averages as (SELECT Region, Year, AVG(Value) as avg_mortality_rate_under5
FROM Indicators i
INNER JOIN Country c
ON i.CountryCode=c.CountryCode
AND IndicatorName='Mortality rate, under-5 (per 1,000)'
AND Region!=''
GROUP BY 1, 2)

SELECT Region, Year, MAX(avg_mortality_rate_under5)
FROM all_averages 
GROUP BY 1
ORDER BY 2

 * sqlite:///indicators.sqlite
Done.


Region,Year,MAX(avg_mortality_rate_under5)
East Asia & Pacific,1960,112.45714285714287
Europe & Central Asia,1960,51.6
Latin America & Caribbean,1960,152.17142857142858
North America,1960,31.35
Middle East & North Africa,1963,198.21333333333334
South Asia,1963,254.8714285714286
Sub-Saharan Africa,1964,244.6166666666667


For all regions, the years with highest mortality rate for children under 5 were in the 60s, which indicates that this issue has improved over time. Let's compare the values in 1960 to 2014.

In [14]:
%%sql

with avg_1960 as (
    SELECT Year, Region, AVG(Value) as avg_mortality_rate_under5_1960
FROM Indicators i
INNER JOIN Country c
ON i.CountryCode=c.CountryCode
AND IndicatorName='Mortality rate, under-5 (per 1,000)'
AND Year='1960' AND Region!=''
GROUP BY 1, 2
ORDER BY 3 DESC)
,

avg_2014 as (
    SELECT Year, Region, AVG(Value) as avg_mortality_rate_under5_2014
FROM Indicators i
INNER JOIN Country c
ON i.CountryCode=c.CountryCode
AND IndicatorName='Mortality rate, under-5 (per 1,000)'
AND Year='2014'
GROUP BY 1, 2
ORDER BY 3 DESC)

SELECT a.Region, avg_mortality_rate_under5_1960, avg_mortality_rate_under5_2014, avg_mortality_rate_under5_2014-avg_mortality_rate_under5_1960 as change
FROM avg_1960 a
LEFT JOIN avg_2014 b
ON a.Region=b.Region
ORDER BY 4 DESC



 * sqlite:///indicators.sqlite
Done.


Region,avg_mortality_rate_under5_1960,avg_mortality_rate_under5_2014,change
North America,31.35,5.85,-25.5
Europe & Central Asia,51.6,9.313725490196076,-42.286274509803924
East Asia & Pacific,112.45714285714287,25.66206896551724,-86.79507389162563
Latin America & Caribbean,152.17142857142858,19.67878787878788,-132.4926406926407
Sub-Saharan Africa,241.556,77.17083333333333,-164.3851666666667
Middle East & North Africa,190.13333333333335,18.990476190476194,-171.14285714285717
South Asia,240.42,44.6875,-195.7325


Comparing the earliest timepoint of the dataset to the most recent, we can see all regions have suffered a decrease in  mortality rate for children under 5. The regions of South Asia, Middle East & North Africa and Sub-Saharan Africa experienced the greatest reduction, nevertheless, they still display some of the highest values for this indicator in 2014.

Even though annual % of population growth **decreased** from 1960 to 2014 for all regions except Sub-Saharan Africa, the mortality rate for children under 5 for **did not increase** in that same period.

This is not surprising. Mortality under 5 is one of many factors that influence population size and we cannot expect for it to solely account for population growth variation over the years.